In [214]:
#Librerias
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
from xml.dom import minidom
from os import listdir 
import re
import numpy as np
import math

In [215]:
#Leer documentos
stop = open("stopWord.txt", "r", encoding="utf-8")
stop_words=stop.readlines()
docs=open("text.txt", "r", encoding="utf-8")
docs=docs.readlines()
steamer=[ ["estudiar","estudiamos","estudio"], ["similitud","similitudes"],
        ["usa","usando"],["modelo","modelos","modelamiento"]]

In [216]:
def remove_StopWords(arrayText):
    newArrayText=[]  
    for txt in arrayText:
        txt=txt+"\n"
        if txt not in stop_words:
            txt=txt.rstrip("\n")
            bol=True
            for arr in steamer:
                if txt in arr:
                    bol=False
                    newArrayText.append(arr[0])
                    break
            if(bol):
                newArrayText.append(txt.rstrip("\n"))              
    return newArrayText

In [217]:
#Procesamiento de un texto utilizando las funciones de Gensim
#Y quitando simbolos
def process(text):     
    stem=re.findall(r'[\w]+', text.lower() ) 
    newStem=remove_StopWords(stem) 
    return newStem

In [218]:
#Crear Diccionario
docDict=[]
for txt in docs:
    docDict.append(process(txt))
dictionary=corpora.Dictionary(docDict)

In [219]:
#Clase para crear corpus teniendo encuenta el diccionario
class MyCorpus(object):
    def __iter__(self):
        for txt in docs:
            yield dictionary.doc2bow(process(txt))        

In [220]:
corpus_memory=MyCorpus()
#Guardar Corpus
corpora.MmCorpus.serialize('corpus.mm',corpus_memory)
#Cargar Corpus
corpus=corpora.MmCorpus('corpus.mm')
#Crear tfidf del corpus
tfidf=models.TfidfModel(corpus)
#Guardar tfidf
#tfidf.save("tfidfCorpus.mm") 
index=similarities.MatrixSimilarity(tfidf[corpus])

In [221]:
#PUNTO 3 CALCULAR SIMILITUD
query="modelo de bolsa de estudio"
query_doc_bow=dictionary.doc2bow(process(query))  
sims=list(index[tfidf[query_doc_bow]])
listSims=list((enumerate(sims))) 
sortSims=sorted(listSims,key=lambda pair: pair[1],reverse=True)
print("Top documentos")
for dat in sortSims:
    id=dat[0]+1
    print("El documento {} tiene una similitud de {} ".format(id,dat[1]))
 

Top documentos
El documento 3 tiene una similitud de 0.5809503793716431 
El documento 4 tiene una similitud de 0.2632814347743988 
El documento 1 tiene una similitud de 0.2360454499721527 
El documento 7 tiene una similitud de 0.21359914541244507 
El documento 6 tiene una similitud de 0.19569028913974762 
El documento 9 tiene una similitud de 0.12573328614234924 
El documento 2 tiene una similitud de 0.0 
El documento 5 tiene una similitud de 0.0 
El documento 8 tiene una similitud de 0.0 
El documento 10 tiene una similitud de 0.0 


In [222]:
#Relevantes
relevantesQuery=[6,7,10,1] #Del query
relevante=[]
for a in sortSims:
    id=a[0]+1
    if id in relevantesQuery:
        relevante.append(1)
    else:
        relevante.append(0)
print(relevante)

[0, 0, 1, 1, 1, 0, 0, 0, 0, 1]


In [223]:
#Función para llenar la matriz del query con Revelante
# P@K y R@K
def rPK_RK(arrayQuery):
    mQuery=[]
    relevantes=sum(arrayQuery)
    n=0
    for i in arrayQuery: 
        mQuery.append([])
        mQuery[n].append(i)# Relevante
        if(n==0):
            mQuery[n].append(i) #Relevante inicial acumulativo          
        else:
            mQuery[n].append(i+mQuery[n-1][1]) #Relevante acumulativo
        mQuery[n].append( mQuery[n][1]/relevantes ) # R@K
        mQuery[n].append(mQuery[n][1]/(n+1) ) # P@K
        n+=1
    return mQuery

In [224]:
mQuery1=np.array(rPK_RK(relevante))
print("Query 1\n","isRelevant sumRelevant  R@K       P@K ")
print(mQuery1,"\n")

Query 1
 isRelevant sumRelevant  R@K       P@K 
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [1.         1.         0.25       0.33333333]
 [1.         2.         0.5        0.5       ]
 [1.         3.         0.75       0.6       ]
 [0.         3.         0.75       0.5       ]
 [0.         3.         0.75       0.42857143]
 [0.         3.         0.75       0.375     ]
 [0.         3.         0.75       0.33333333]
 [1.         4.         1.         0.4       ]] 



In [232]:
print("Respuestas")
print("P@5= {} R@5= {} F@5 {} ".format(mQuery1[4][3],mQuery1[4][2], (2*mQuery1[4][3]*mQuery1[4][2])/(mQuery1[4][3]+mQuery1[4][2])  ))
print("P@3= {} R@3= {} F@3 {} ".format(mQuery1[2][3],mQuery1[2][2],(2*mQuery1[2][3]*mQuery1[2][2])/(mQuery1[2][3]+mQuery1[2][2]) ))
print("")

Respuestas
P@5= 0.6 R@5= 0.75 F@5 0.6666666666666665 
P@3= 0.3333333333333333 R@3= 0.25 F@3 0.28571428571428575 



In [226]:
def dfGainDCGI(query_rel):
    mat=[]
    n=0
    for i in query_rel:
        mat.append([])
        mat[n].append(i) #REl_i
        mat[n].append(1/(math.log(max(n+1,2), 2))) #discount_factor
        mat[n].append(i*mat[n][1])
        if(n==0):
            mat[n].append(mat[n][2]) #DCG_i
        else:
            mat[n].append(mat[n][2]+mat[n-1][3]) #DCG_i
        n+=1
    return mat

In [227]:
def NDCG(matQ,matQN):
    row=len(matQ)
    col=len(matQ[0])
    return matQ[row-1][col-1]/matQN[row-1][col-1] 

In [228]:
escalaR={ 6:4,7:3,10:3,1:5 }
vectorEscala=[] 
for a in sortSims:
    id=a[0]+1
    if id in escalaR:
        vectorEscala.append(escalaR[id])
    else:
        vectorEscala.append(0)
vectorEscala

[0, 0, 5, 3, 4, 0, 0, 0, 0, 3]

In [231]:
dcg=np.array(dfGainDCGI( vectorEscala ))
dcgN=np.array(dfGainDCGI( sorted( vectorEscala ,reverse=True)))
#print(dcg,"\n","\n",dcgN)

ndcg=NDCG(dcg,dcgN)
print("NDCG: De los 10 documentos es:",ndcg) 

NDCG: De los 10 documentos es: 0.5874742831480317
